# Contents
- [Purpose](#Purpose)
- [Library import](#Library-import)
- [Input params](#Input-params)
- [Function definitions](#Function-definitions)
- [EO Data Conversion Tool resampling](#EO-Data-Conversion-Tool-resampling)
- [Join EO Data Conversion Tool files](#Join-EO-Data-Conversion-Tool-files)
- [GCloud and manifest file](#GCloud-and-manifest-file)
- [Build manifest and ingest files into GEE](#Build-manifest-and-ingest-files-into-GEE)
***

# Purpose
[Return to the "Table of contents"](#Contents)

This notebook does the remapping and ingestion of swath images found in   
https://code.earthengine.google.com/65d4c2d66a27721287f4f6d80e0e614b


***

# Library import   

[Return to the "Table of contents"](#Contents)

Import all required modules below for the Level-2 resampling.  

---

In [1]:
!pip install netcdf4 pyresample gdal h5py pyproj

In [2]:
import os
import subprocess
import time
from pathlib import Path
from pprint import pprint

import json
import numpy as np
from osgeo import osr, gdal

Local imports

In [3]:
import sys
sys.path.append('utils')

In [4]:
from utils.swathresample import SwathResample

# Input params
[Return to the "Table of contents"](#Contents)

Define the input file (netCDF4 or HDF5), output projection for the GeoTIFF file and the area id.  
We can also define here the [Google cloud bucket](https://cloud.google.com/storage/docs/gsutil), the [Earth Engine Asset](https://developers.google.com/earth-engine/guides/asset_manager).  The EE asset is where GeoTIFF file is uploaded to.

Sample files are included `sample_data` folder or can be downloaded directly form the providers
1. https://oceandata.sci.gsfc.nasa.gov/ob/getfile/A2022125035500.L2_LAC_OC.nc
2. https://gportal.jaxa.jp/download/standard/GCOM-C/GCOM-C.SGLI/L2.OCEAN.IWPR/3/2022/05/03/GC1SG1_202205030152F05810_L2SG_IWPRQ_3000.h5

---

1. The case of MODIS/Aqua sample file

In [5]:
# Smaple SGLI/GCOM-C files
INPUT_FILE = (Path('sample_data/A2022125035500.L2_LAC_OC.nc'), 
              Path('sample_data/GC1SG1_202205030152F05810_L2SG_IWPRQ_3000.h5'))
# Target projection
PROJ_NAME = 'laea'
# Projection area ID
# Output Path
OPATH = Path('result').absolute()
if not OPATH.is_dir():
    OPATH.mkdir(parents=True)
# GeoTIFF output
TRG_TIFF = (OPATH.joinpath('A2022125035500.L2_LAC_OC.tif'), 
            OPATH.joinpath('GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_py.tif'))
# projection id
AREA_ID = 'nowpap_region'
# Google Cloud Bucket id
BUCKET = 'gs://YOUR_BUCKET_NAME'
# Giving a name of an existing asset will result in error
ASSET_ID = ''
# Asset name should not contain dots (.)
ASSET_NAME = 'A2022125035500_L2_LAC_OC', 'GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_py'
# list will be updated with files to be uploaded
UPLOAD_TIF = []

# Function definitions
[Return to the "Table of contents"](#Contents)

Operations with Level-2 file 
`SwathResmaple` contains the following helper functions
1. `SwathResmaple.resample` - for swath resampling, accepts, data, cutoff and sequence number
2. `SwathResmaple.scale` - for data scaling into integer, accepts data and varname
3. `SwathResmaple.open` - open a new GeoTIFF file in write mode, accepts number os bands
4. `SwathResmaple.append` - append var(i) in the opened GeoTIFF, accept band sequence number and varname
5. `SwathResmaple.translate` - GDAL translate the created GeoTIFF
6. `SwathResmaple.close` - closes file, both netCDF4/HDF5 and GeoTIFF

In [6]:
def filt_keys(key):
    keep = 'chlor_a', 'CHLA', 'QA_flag', 'l2_flags'
    return key in keep

In [7]:
for src, dst in zip(INPUT_FILE, TRG_TIFF):
    with SwathResample(src, trg_tif=dst, srs=PROJ_NAME, area_id=AREA_ID) as fid:
    
        # get the spatial resolution from file metadata
        roi = fid.spatial_resolution()
        print(f'SpatialResolution: {roi}')
        # in the case of SGLI pass two files, since CHL and Rrs are in separate files
        keys = list(filter(filt_keys, fid.get_keys()))

        # joined = '\n\t'.join(keys)
        # print(f'Mapping variables\n\t{joined}')

        # open GeoTIFF file
        fid.open(bands=len(keys))

        # to avoid memory errors, especially with SGLI 250 m, process one variable at a time
        for i, key in enumerate(keys):
            # Read data from file
            sds = fid.get_data(key=key)
            # Pyresample, use twice the resolution. 
            # compansate for the decrease in resolution towards swath edge
            res = fid.resample(data=sds, i=i, roi=roi*2, key=key)
            # Scale data to int
            fid.data = fid.scale(key=key, data=res)
            # output result in GeoTIFFe
            fid.append(band=i+1, key=key)

        # translate the tif
        fid.translate()
        # There is temporary file created internally, remove
        tmp_file = fid.tmp_tif
    tmp_file.unlink(missing_ok=True)
    !gdalinfo {dst}
    UPLOAD_TIF.append(dst)

Rounding shape to (2284, 2835) and resolution from (1001.0, 1001.0) meters to (1000.9958309752008, 1000.9164345319438) meters
C:\Users\Eligio\AppData\Roaming\Python\Python39\site-packages\pyproj\crs\crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)


SpatialResolution: 1000.0
resampling... 
	 0:   chlor_a | Elapsed 2.672 sec
	 1:  l2_flags | Elapsed 2.426 sec
gdal_translate -of GTiff -r nearest -ot Int32 -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co compress=lzw C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\A2022125035500.L2_LAC_OC_tmp.tif C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\A2022125035500.L2_LAC_OC.tif
Driver: GTiff/GeoTIFF
Files: C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\A2022125035500.L2_LAC_OC.tif
Size is 2835, 2284
Coordinate System is:
PROJCRS["laea",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["Lambert Azimuthal Equal Area",
        METHOD["Lambert Azimuthal Equal Area",
            ID[

Rounding shape to (8621, 8040) and resolution from (251.0, 251.0) meters to (250.99112192356145, 250.97432778700153) meters
C:\Users\Eligio\AppData\Roaming\Python\Python39\site-packages\pyproj\crs\crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)


SpatialResolution: 250.0
resampling... 
	 0:      CHLA | Elapsed 42.121 sec
	 1:   QA_flag | Elapsed 38.454 sec
gdal_translate -of GTiff -r nearest -ot Int32 -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co compress=lzw C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_py_tmp.tif C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_py.tif
Driver: GTiff/GeoTIFF
Files: C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_py.tif
Size is 8040, 8621
Coordinate System is:
PROJCRS["laea",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["Lambert Azimuthal Equal Area",
 

# [EO Data Conversion Tool](https://gportal.jaxa.jp/gpr/information/tool?lang=en) resampling
[Return to the "Table of contents"](#Contents)

- Use the EO Data Conversion Tool for resampling  
Applies to GCOM-C
> First download and install the Windows DCT tool from https://gportal.jaxa.jp/gpr/information/tool?lang=en  
> Next run the commands below

In [8]:
of = Path('result').absolute() # output path
for key in ('CHLA', 'QA_flag'):
    cmd = f'SGLI_geo_map_win {INPUT_FILE[-1]} -d Image_data/{key} -r 0 -o {of} -n 65535'
    print(os.popen(cmd).read())

========================== Input parameters ==========================
Input file              = sample_data\GC1SG1_202205030152F05810_L2SG_IWPRQ_3000.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = Default (7.5sec/15sec/30sec)
Resampling method       = NN
Data selection          = Image_data/CHLA
GDAL_NODATA Tag         = 65535
Output directory        = C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result
product_type = L2/scene
rtype = Q
x range =  123.2083333333,  146.4000000000
y range =   27.0416666667,   46.4583333333
pid = GC1SG1_202205030152F05810_L2SG_IWPRQ_3000
map = Lat/Lon
dataset = Image_data/CHLA
datatype = unsigned short
max_valid_DN = 65534
isize = 7820/5000
output size = 9320, 11132
block size = 20, 20
grid size = 467, 558
    0/ 9340
 2000/ 9340
 4000/ 9340
 6000/ 9340
 8000/ 9340
data resampling...
    0/ 9320
 2000/ 9320
 4000/ 9320
 6000/ 9320
 8000/ 9320
output = [C:\Users\Eligio\Documents\NPEC\Python\eeuplo



# Join [EO Data Conversion Tool](https://gportal.jaxa.jp/gpr/information/tool?lang=en) files
[Return to the "Table of contents"](#Contents)

- Combine the files created by the Earth Observation Data Conversion Tool into a single GeoTIFF
> This step is not important. We can combine these files directly in the ingestion manifest.   
But I prefered to do it here and GDAL translate combined image. 
- This operation will also copy attributes of the original .h5 file

In [9]:
dst=OPATH.joinpath('GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_dct.tif')
src=(OPATH.joinpath('GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_CHLA.tif'),
     OPATH.joinpath('GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_QA_flag.tif'))

with SwathResample(INPUT_FILE[1], trg_tif=dst, srs='lonlat', area_id=AREA_ID) as fid:
    keys = 'CHLA', 'QA_flag'
    # open GeoTIFF file
    fid.copy_tif(bands=len(keys), src=src[0])
        
    for i, (f, key) in enumerate(zip(src, keys)):
        # Read data from file
        sds = fid.get_band(file=f)
        print(f'{i} | {f.name}: {key} | FV: {sds.fill_value}')
        # Scale data to int
        if 'CHLA' in f.name:
            # Note that CHLA is digital number. 
            # We need to scale factor the original data before scaling with the unified value for both files.
            sds *= 0.001600
        fid.data = fid.scale(key=key, data=sds)
        # output result in GeoTIFFe
        fid.append(band=i+1, key=key)
        
        # There is temporary file created internally
        tmp_file = fid.tmp_tif
        
    # translate the tif
    fid.translate()
    # remove temporary file 
    tmp_file.unlink(missing_ok=True)
    UPLOAD_TIF.append(dst)
!gdalinfo {dst}

0 | GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_CHLA.tif: CHLA | FV: 65535.0
1 | GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_QA_flag.tif: QA_flag | FV: 65535
gdal_translate -of GTiff -r nearest -ot Int32 -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co compress=lzw C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_dct_tmp.tif C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_dct.tif
Driver: GTiff/GeoTIFF
Files: C:\Users\Eligio\Documents\NPEC\Python\eeupload\ee-oc-data-ingestion\result\GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_dct.tif
Size is 11132, 9320
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
   

# GCloud and manifest file
[Return to the "Table of contents"](#Contents)

1. Send file to Google Cloud bucket (or file can also be [uploaded](https://developers.google.com/earth-engine/guides/image_upload) directly into GEE)
2. Build ingestion manifest file 
3. Send to GEE using Earth Engine [command-line tools](https://developers.google.com/earth-engine/guides/command_line#upload) 

`build_manifest` accepts the GeoTIFF file, asset_id, asset_name and uri (location of the file in the Gcloud), builds the manifest file)

In [10]:
def build_manifest(file, asset_id, asset_name, uri):
    """
    Writes earth engine manifest file for data upload
    manifest file name
    """

    dataset = gdal.Open(str(file), gdal.GA_ReadOnly)
    attrs = dataset.GetMetadata()
    update = attrs.update

    data = {
        "name": asset_id,
        "tilesets": [
            {'id': asset_name,
             "sources": [
                 {"uris": [uri]}
             ]}
        ],
        "start_time": attrs.pop("time_coverage_start"),
        "end_time": attrs.pop("time_coverage_end"),
        'bands': [],
        "properties": attrs
    }
    append = data['bands'].append

    for count in range(dataset.RasterCount):
        band = dataset.GetRasterBand(count + 1)
        varname = band.GetDescription()

        var_attrs = band.GetMetadata()
        update(var_attrs)
        if varname in ('QA_flag', 'l2_flags'):
            append({"id": varname,
                    'tileset_id': asset_name,
                    "pyramidingPolicy": "SAMPLE",
                    "tileset_band_index": count})
        else:
            novalue = band.GetNoDataValue()
            append({"id": varname,
                    'tileset_id': asset_name,
                    "tileset_band_index": count,
                    "pyramidingPolicy": "MEAN",
                    "missing_data": {
                        "values": [float(novalue)]}
                    })
    
    fid = Path(file).name.strip(Path(file).suffix)
    manifest = Path(f'result/{fid}_manifest.json').absolute()
    pprint(data)
    with open(manifest, 'w') as mfs:
        json.dump(data, mfs)
    return manifest

In [11]:
UPLOAD_TIF

[WindowsPath('C:/Users/Eligio/Documents/NPEC/Python/eeupload/ee-oc-data-ingestion/result/A2022125035500.L2_LAC_OC.tif'),
 WindowsPath('C:/Users/Eligio/Documents/NPEC/Python/eeupload/ee-oc-data-ingestion/result/GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_py.tif'),
 WindowsPath('C:/Users/Eligio/Documents/NPEC/Python/eeupload/ee-oc-data-ingestion/result/GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_dct.tif')]

In [12]:
# -------------
# GCloud upload
# -------------
for trg in UPLOAD_TIF:
    uri = f'{BUCKET}/{trg.name}'
    cmd = f'gsutil cp {trg.name} {BUCKET}'
    info = f'{"-" * len(cmd)}'
    print(f'{info}\n{cmd}\n{info}')
    # uncomment below if bucket is defined
    # !gsutil cp {trg} {BUCKET}

------------------------------------------------------------
gsutil cp A2022125035500.L2_LAC_OC.tif gs://YOUR_BUCKET_NAME
------------------------------------------------------------
--------------------------------------------------------------------------------
gsutil cp GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_py.tif gs://YOUR_BUCKET_NAME
--------------------------------------------------------------------------------
---------------------------------------------------------------------------------
gsutil cp GC1SG1_202205030152F05810_L2SG_IWPRQ_3000_dct.tif gs://YOUR_BUCKET_NAME
---------------------------------------------------------------------------------


# Build manifest and ingest files into GEE

[Return to the "Table of contents"](#Contents)

## Start an authorized session

To be able to upload the file into your Earth Engine asset, you need to authenticate as you when you make the request. 

In [13]:
import ee

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AdQt8qivF5i2npyci_N13fAI0TcF1e1kcFPtXFWR5JmVMhm0KB85U4duXdc

Successfully saved authorization token.


In [14]:
# --------------
# build manifest
# --------------
for trg in UPLOAD_TIF:
    asset_name = '_'.join(trg.name.split(trg.suffix)[0].split('.'))
    uri = f'{BUCKET}/{trg.name}'
    manifest = build_manifest(file=trg, 
                              asset_id=f'{ASSET_ID}/{asset_name}', 
                              asset_name=asset_name,
                              # asset_name=ASSET_NAME, 
                              uri=uri)
    # -----------
    # Ingest file 
    # -----------
    cmd = f'earthengine upload image --manifest {Path(manifest).name}'
    info = f'{"-" * len(cmd)}'
    print(f'{info}\n{cmd}\n{info}')
    # uncomment as necessary
    # !earthengine upload image --manifest {manifest}
    print()
print('done..!!')

{'bands': [{'id': 'chlor_a',
            'missing_data': {'values': [-32767.0]},
            'pyramidingPolicy': 'MEAN',
            'tileset_band_index': 0,
            'tileset_id': 'A2022125035500_L2_LAC_OC'},
           {'id': 'l2_flags',
            'pyramidingPolicy': 'SAMPLE',
            'tileset_band_index': 1,
            'tileset_id': 'A2022125035500_L2_LAC_OC'}],
 'end_time': '2022-05-05T03:59:59.583Z',
 'name': '/A2022125035500_L2_LAC_OC',
 'properties': {'AREA_OR_POINT': 'Area',
                'Conventions': 'CF-1.6 ACDD-1.3',
                'cdm_data_type': 'swath',
                'chlor_a__FillValue': '-32767.0',
                'chlor_a_add_offset': '0.001',
                'chlor_a_long_name': 'Chlorophyll Concentration, OCI Algorithm',
                'chlor_a_reference': 'Hu, C., Lee Z., and Franz, B.A. (2012). '
                                     'Chlorophyll-a algorithms for '
                                     'oligotrophic oceans: A novel approach '
     